In [15]:
import cv2
import glob
import numpy as np

In [16]:
images = []
for img in glob.glob(".\\house\\*.jpg"):
    image = cv2.imread(img)
    images.append(image)

In [17]:
exposure_times = np.array([100, 250, 500, 800], dtype=np.float32)

alignMTB = cv2.createAlignMTB()
alignMTB.process(images, images)

In [18]:
# Merge exposures to HDR image
merge_debevec = cv2.createMergeDebevec()
hdr_debevec = merge_debevec.process(images, times=exposure_times.copy())

merge_robertson = cv2.createMergeRobertson()
hdr_robertson = merge_robertson.process(images, times=exposure_times.copy())

# Tonemap HDR image
tonemap1 = cv2.createTonemap(gamma=2.2)
res_debevec = tonemap1.process(hdr_debevec.copy())

# Exposure fusion using Mertens
merge_mertens = cv2.createMergeMertens()
res_mertens = merge_mertens.process(images)

# Convert datatype to 8-bit and save
res_debevec_8bit = np.clip(res_debevec*255, 0, 255).astype('uint8')
res_mertens_8bit = np.clip(res_mertens*255, 0, 255).astype('uint8')

cv2.imwrite(r"ldr_debevec.jpg", res_debevec_8bit)
cv2.imwrite(r"fusion_mertens.jpg", res_mertens_8bit)

True